In [137]:
#import import_ipynb
#from Main import Board, Coord
from collections import namedtuple
import random


def Override(f):
    return f

Coord = namedtuple("Coord", ["X", "Y"]) 


In [7]:
#make Piece a subclass of object so we can use things like Super
class Piece(object):
    """A class to define the individual pieces, their descriptions, type, shape, orientation, and color"""
    
    IDStart = 9000
    IDEnd = 9010
    
    IDGenerator = list(range(IDStart, IDEnd))
    
    
    def __init__(self, pShape, pType="None", pOri="1", pColor="NONE", pDesc=""): #idk if there's a purpose to the p's
        
        self.shape = [Coord(square.X, -square.Y) for square in pShape] #we mirror the shape since 
                            #representation when printing is flipped. the rotate function is also changed accordingly.
        self.type = "Basic"
        self.ori = pOri
        self.desc = pDesc
        self.color = pColor
        self.rotations = {1:self.shape}
        self.coordinates = [] #I don't actually know what I'm using these coordinates for, maybe later?
        self.origin = Coord(0,0)
        self.__CacheRotations__()
        self.image = None
        
        self.upgraded = False
        self.ID = self.__AssignID__()
        
        self.value = 0
        
        
    #def __iter__(self):
    #    return self
    # 
    #def __next__(self):
    def __getitem__(self, index):
        return self.shape[index]
    
    def __CacheRotations__(self):
        for ori in range(2, 5):
            self.rotations[ori] = self.Rotate(cache=True)
        self.SetOrientation(1)
        
    
    @classmethod
    def __AssignID__(cls):
        newID = random.choice(cls.IDGenerator)
        cls.IDGenerator.remove(newID)
        return newID
    
    @classmethod
    def ResetAllPieceIDs(cls):
        """Reset the list of ID's that new pieces are picked from."""
        cls.IDGenerator = list(range(cls.IDStart, cls.IDEnd))
        #need to also clear all existing pieces probably
        
    def GetImage(self):
        return self.image
    
    def SetImage(self, img):
        self.image = img
    
    def GetID(self):
        return self.ID
    
    def GetColor(self):
        return self.color
    
    def SetDescription(self, pDesc):
        self.desc = pDesc
        
    def GetOrientation(self):
        return self.ori
    
    def SetOrientation(self, ori):
        self.ori = ori
        self.shape = self.rotations[ori]
        
    def GetCoordinates(self):
        return self.coordinates
                
    def SetCoordinates(self, listCoord):
        """The first Coordinate should always be the origin of the piece. May make that explicit later."""
        self.coordinates = listCoord
        self.origin = listCoord[0]
        
    def Upgrade(self):
        self.upgraded = True
        
    def Downgrade(self):
        self.upgraded = False
        
    def Rotate(self, cache=False):
        """Rotate the mirrored piece coordinates 90deg clockwise. To get a true 90d rotate:newCoord = Coord(oldCoord.Y, -oldCoord.X) """
        if cache:
            newRotation = []
            for oldCoord in self.shape:
                newCoord = Coord(-oldCoord.Y, oldCoord.X)
                newRotation.append(newCoord)

            self.shape = newRotation
            return newRotation
            #I wanted to use itertools.cycle here but decided against it.
        else:
            self.ori += 1
            if self.ori >4: 
                self.ori = 1
            self.shape = self.rotations[self.ori]
                
            
    
    
    #####Point Based Functions Next####
    def Activate(self, value, prevProcessList, nextProcessList, board):
        """Given a number value, the previously processed pieces in the active line, 
           and the upcoming pieces in the active line, calculate and return a new value."""
        
        #Should be null since this is a skeleton class
        
        print(prevProcessList, self.ID, nextProcessList)
        #return value + ((self.ID - 9000)//100) 
        
        return value
        
         

In [139]:
class BluePiece(Piece):
    """Extends the basics Piece class, as a simple valued piece object."""
    def __init__(self, *args, pValue=10, **kwargs):
        super().__init__(*args, **kwargs)
        self.type = "VALUE"
        self.value = pValue
        self.color = "BLUE"
        
    def Activate(self, value, prevProcessList, nextProcessList, board):
        #If not upgraded:
        if not self.upgraded:
            if self.ID in prevProcessList:
                return value
            else:
                return value + self.value
        #If it is upgraded:
        elif self.upgraded:
            return value + self.value
        

In [140]:
class GreenPiece(Piece):
    """Extends our basic Piece class, giving it a color:GREEN, type:MULTIPLIER, and changes the activate fn."""
    def __init__(self, *args, pMultiplier=2, **kwargs):
        super().__init__(*args, **kwargs)
        self.type = "MULTIPLIER"
        self.multiplier = pMultiplier
        self.color = "GREEN"
    
    
    #def __init__(self, pShape, pType="None", pOri="1", pColor="BLUE", pDesc="", pMultiplier=2): #idk if there's a purpose to the p's
        
    #    self.shape = [Coord(square.X, -square.Y) for square in pShape] #we mirror the shape since 
                            #representation when printing is flipped. the rotate function is also changed accordingly.
    #    self.type = "MULTIPLIER"
    #    self.pMultiplier = pMultiplier
    #    self.ori = pOri
    #    self.desc = pDesc
    #    self.color = "GREEN"
    #    self.rotations = {1:self.shape}
    #    self.coordinates = []
        
    #    self.upgraded = False
        
    #    self.__CacheRotations__()
        #print(self.rotations)
        
        
    #    self.ID = self.__AssignID__()
        
    @Override #doesn't actually check anything, just pretty.
    def Activate(self, value, prevProcessList, nextProcessList, board):
        """Given a number value, multiplies the value by this piece's designated amount and returns the new value"""
        
        print(prevProcessList, self.ID, nextProcessList)
        
        #Maybe we should change this to two separate functions that get called, depending if upgraded or not
        if not self.upgraded:
            if self.ID in prevProcessList:
                return value
            else:
                return value * self.multiplier
        elif self.upgraded:
            #Compound multiplication for an upgraded piece, eg. if this piece covers two spots on the 
            #active line, then it multiplies twice.
            return value * self.multiplier

In [5]:
class YellowPiece(Piece):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.type = "UPGRADE"
        self.color = "YELLOW"
        self.upgradeNext = True
        
    def Activate(self, value, prevProcessList, nextProcessList, board):
        #TODO
        pass #board.
        
        return value

In [ ]:
class MirrorPiece(Piece):
    """Reflects the direction of the active line after hitting this piece"""
    #I wonder if we would have to change the IDs of all the pieces for the new mirrored part
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        self.type = "MIRROR"
        self.color = "MIRROR"
    
    def MirrorNextPieces(self, prevProcessList, nextProcessList, board):
        #TODO
        pass

In [ ]:
w = YellowPiece([Coord(0,0)])


In [ ]:
q = GreenPiece([Coord(1, 1)])

In [ ]:
print(w.GetID(), q.GetID())
Piece.IDGenerator
q.shape
w.value